In [ ]:
from waveletae.training.toydata import generate_multi_sine

def generate_toy_data(T=600, n=4, sparsity=0.5, r=128, timing_mode="gaussian", use_dwt=True):
    signals, masks, _ = generate_multi_sine(T=T, n=n, sparsity=sparsity)

    resampled_vals = []
    resampled_time = []
    for i in range(n):
        from waveletae.preprocessing.interpolation import linear_interpolate
        from waveletae.preprocessing.timing import timing_signal

        vals = linear_interpolate(signals[i], masks[i], r)
        times = timing_signal(masks[i], mode=timing_mode, r=r)

        resampled_vals.append(vals)
        resampled_time.append(times)

    X = np.stack(resampled_vals + resampled_time, axis=0)

    if use_dwt:
        from waveletae.preprocessing.wavelet import apply_dwt
        X = apply_dwt(X)

    return X
